# Proyecto Acrobot con NEAT


In [7]:
%pip install neat-python
#!pip install neat-python

     ---------------------------------------- 44.2/44.2 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
%pip install coppeliasim-zmqremoteapi-client
#!pip install coppeliasim-zmqremoteapi-client

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py install for cbor: started
  Running setup.py install for cbor: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  DEPRECATION: cbor is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
%pip install cbor
#!pip install cbor

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
%pip install numpy
#!pip install numpy
%pip install time
#!pip install time
%pip install pandas
#!pip install pandas


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time

[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     --------------------------------------- 10.6/10.6 MB 73.0 MB/s eta 0:00:00
     ------------------------------------- 502.5/502.5 kB 32.8 MB/s eta 0:00:00
     ---------------------------------------- 341.8/341.8 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import numpy as np
import time

from python.zmqRemoteApi import RemoteAPIClient


class Coppelia():

    def __init__(self):
        print('*** connecting to coppeliasim')
        client = RemoteAPIClient()
        self.sim = client.getObject('sim')

    def start_simulation(self):
        # print('*** saving environment')
        self.default_idle_fps = self.sim.getInt32Param(self.sim.intparam_idle_fps)
        self.sim.setInt32Param(self.sim.intparam_idle_fps, 0)
        self.sim.startSimulation()

    def stop_simulation(self):
        # print('*** stopping simulation')
        self.sim.stopSimulation()
        while self.sim.getSimulationState() != self.sim.simulation_stopped:
            time.sleep(0.1)
        # print('*** restoring environment')
        self.sim.setInt32Param(self.sim.intparam_idle_fps, self.default_idle_fps)
        print('*** done')

    def is_running(self):
        return self.sim.getSimulationState() != self.sim.simulation_stopped


In [2]:
import sim
import numpy as np
import pandas as pd
import math

In [3]:
class Acrobot():
    
    num_sonar = 16
    ang_max = np.pi
    joint_pos=0
    joint2_handle=-1
    dummy_handle=-1
    joint1_handle=-1
    link1_handle=-1
    joint2_handle=-1
    link2_handle=-1
    def __init__(self, sim, robot_id, use_camera=False, use_lidar=False):
        self.sim = sim
        print('*** getting handles', robot_id)
        #self.joint1 = self.sim.getObject(f'/{robot_id}/Acrobot/Joint1')
        #self.joint2 = self.sim.getObject(f'/{robot_id}/Acrobot/Joint1/Link1/Joint2')
        
        self.dummy_handle = self.sim.getObject(f'/{robot_id}/Acrobot/Joint1/Link1/Joint2/Link2/Dummy')
        self.joint1_handle = self.sim.getObject(f'/{robot_id}/Acrobot/Joint1')
        self.joint2_handle = self.sim.getObject(f'/{robot_id}/Acrobot/Joint1/Link1/Joint2')
        self.link1_handle = self.sim.getObject(f'/{robot_id}/Acrobot/Joint1/Link1')
        self.link2_handle = self.sim.getObject(f'/{robot_id}/Acrobot/Joint1/Link1/Joint2/Link2')


    def set_speed(self, speed):
        self.sim.setJointTargetVelocity(self.joint2_handle, speed)
    def set_pos_joint(self,pos):
        pos=pos*np.pi/180
        self.sim.setJointTargetPosition(self, pos,  [])
    def set_joint_torque(self,torque):
        self.sim.setJointTargetForce( self.joint2_handle, torque, True)
    def get_joint_torque(self):
        return self.sim.getJointForce( self.joint2_handle)
    def get_handle(self):
        return self.joint2_handle

    def get_pos_joint2(self):
        self.joint_pos=self.sim.getJointPosition(self.joint2_handle)*180/np.pi
        return self.joint_pos

    def get_pos_joint1(self):
         return self.sim.getJointPosition(self.joint1_handle) * 180 / np.pi

    def get_pos_dummy(self):
        return self.sim.getObjectPosition(self.dummy_handle,self.sim.handle_world)
         
    def get_velocity_joint(self):
        return  self.sim.getJointVelocity(self.joint2_handle)
    def get_velocity_dummy(self):
        return  self.sim.getObjectVelocity(self.dummy_handle)
    def get_torque_joint(self):
        return self.sim.getJointForce(self.joint2_handle)
    
    def get_dimensiones_link1(self):
        return self.sim.getShapeGeomInfo(self.link1_handle)
    
    def get_dimensiones_link2(self):
        return self.sim.getShapeGeomInfo(self.link2_handle)
    
    def start_motor(self):
        self.sim.setObjectInt32Param(self.joint2_handle,sim.sim_jointintparam_motor_enabled,1)
        #self.sim.setObjectInt32Param(self.joint2_handle,sim.sim_jointintparam_ctrl_enabled,1)



## Conexión

## Obtener Manejadores

In [8]:
def calcular_pos(robot):
    _,_,dimensions1=robot.get_dimensiones_link1()
    _,_,dimensions2=robot.get_dimensiones_link2()
    theta1=robot.get_pos_joint1()
    theta2=robot.get_pos_joint2()
    theta1_r=math.radians(theta1)
    theta2_r=math.radians(theta2)
    L1=dimensions1[0]
    L2=dimensions2[0]
    print("Informacion: ", L1," ",L2," ",theta1," ",theta2)
    '''    if theta2<0:
        theta2+=360
    if theta1<0:
        theta1+=360'''
    x1,y1=L1*math.sin(theta1_r),-L1*math.cos(theta1_r)
    x2,y2=x1+L2*math.sin(theta2_r+theta1_r),y1-L2*math.cos(theta2_r+theta1_r)
    return y2
def main(args=None):
    lista_pos1=[]
    lista_pos2=[]
    coppelia = Coppelia()
    robot = Acrobot(coppelia.sim, 'Cuboid')
    #robot.set_speed(+1.2)
    coppelia.start_simulation()
    robot.start_motor()
    #robot.set_speed(+3.14)
    robot.set_joint_torque(-0.3)
   
    posicion_inicial=calcular_pos(robot)
    print('Pos inicial ', posicion_inicial)
    print("distancia:", 1-calcular_pos(robot))
    print(f'Position Dummy:',robot.get_pos_dummy()[2] )
    print("Velocidad angular: ",robot.get_velocity_joint())
    print("velocidad dummy: ", robot.get_velocity_dummy())
    while (t := coppelia.sim.getSimulationTime()) < 5:
        print(f'Simulation time: {t:.3f} [s]')
        #print(f'Torque joint: ',robot.get_torque_joint())
        lista_pos1.append(robot.get_pos_joint1())
        print(f'Position Dummy:',robot.get_pos_dummy()[2] )
        lista_pos2.append(robot.get_pos_joint2())
        print("Posición : ", calcular_pos(robot))
        #print("Comparacion: ",calcular_pos(robot)+1.23," --- ",robot.get_pos_dummy()[2] )

    coppelia.stop_simulation()
    print("Lista1:", len(lista_pos1),", ",lista_pos1)
    print("Lista2: ", len(lista_pos2),"lista_pos2")

In [11]:
#+2.2275 altura máxima del dummy
#con torque 0.2 no llega a subir
#fitness será el que más tiempo esté el dummy en la posición más alta
#Exito: dejar x segundos el dummy en la posición más alta
#ctrl+w cierra escena en copelia
main()

*** connecting to coppeliasim
*** getting handles Cuboid
Informacion:  0.5   0.5   0.0   0.0
Pos inicial  -1.0
Informacion:  0.5   0.5   0.0   0.0
distancia: 2.0
Position Dummy: 0.20250000000016155
Velocidad angular:  -7.07952580121454
velocidad dummy:  ([-1.1213252548714081e-13, -0.8184170460091411, 0.12460376981731802], [-6.211832898400307e-13, 8.021849609057974e-14, -2.8670123830799294])
Simulation time: 0.050 [s]
Position Dummy: 0.20873018849102745
Informacion:  0.5   0.5   10.942601694547433   -34.413196732901234
Posición :  -0.9495412191225187
Simulation time: 0.100 [s]
Position Dummy: 0.25441818183143394
Informacion:  0.5   0.5   10.942601694547433   -34.413196732901234
Posición :  -0.9495412191225187
Simulation time: 0.100 [s]
Position Dummy: 0.32741629761566937
Informacion:  0.5   0.5   16.50629199201   -53.326978762222424
Posición :  -0.8796517898036718
Simulation time: 0.150 [s]
Position Dummy: 0.32741629761566937
Informacion:  0.5   0.5   16.50629199201   -67.83690495929292

In [ ]:
'''L1=0.5
L2=0.5
theta1=8.50083075711262
theta2=-30.382
#0.7983634206803992
x1,y1=L1*math.sin(theta1),-L1*math.cos(theta1)
print(x1,y1)
x2,y2=x1+(L2*math.sin(theta2+theta1)),y1-(L2*math.cos(theta2+theta1))
print(y2)'''

0.3989933557350547 0.30133752185760093
0.7983167051655288
